In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%%nuclio cmd
pip install kfserving
pip install numpy
pip install sklearn
pip install joblib

  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /igz/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [3]:
import kfserving
import os
import numpy as np
from typing import List, Any

import sklearn
import joblib

In [4]:
class SKModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str):
        super().__init__(name)
        self.name = name
        self.model_filepath = model_dir
        self.model = None


    def load(self):
        model_file = kfserving.Storage.download(self.model_filepath)

        try:
            self.model = joblib.load(model_file)
        except:
            self.model = pickle.load(model_file)

        self.ready = True


    def predict(self, body: List) -> List:

        if not self.ready:
            self.load()

        try:
            # Predict
            predicted_probability = self.model.predict(body)
            return predicted_probability.tolist()

        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [5]:
# nuclio: end-code
from mlrun import new_model_server
import requests

In [6]:
fn = new_model_server('iris-sklearn-advanced', models={'iris_v1': '/User/models/iris_v1/model.sklearn'}, model_class='SKModel')
fn.with_v3io('User','~/') 
print(f'Function yaml (dict):\n{fn.to_dict()}')

Function yaml (dict):
{'kind': 'remote', 'metadata': {'name': 'iris-sklearn-advanced'}, 'spec': {'command': '', 'args': [], 'image': '', 'mode': '', 'description': '', 'volumes': [{'volume': {'flexVolume': {'driver': 'v3io/fuse', 'options': {'accessKey': '460ee949-eaaa-468a-ae26-5fdb9156996e', 'container': 'users', 'subPath': '/admin'}}, 'name': 'fs'}, 'volumeMount': {'name': 'fs', 'mountPath': 'User'}}], 'env': {'SERVING_MODEL_iris_v1': '/User/models/iris_v1/model.sklearn', 'TRANSPORT_PROTOCOL': 'seldon', 'ENABLE_EXPLAINER': 'False', 'MODEL_CLASS': 'SKModel', 'V3IO_USERNAME': 'admin', 'V3IO_ACCESS_KEY': '460ee949-eaaa-468a-ae26-5fdb9156996e', 'V3IO_API': 'v3io-webapi.default-tenant.svc:8081'}, 'config': {'spec.triggers.http': {'kind': 'http', 'maxWorkers': 8, 'attributes': {'ingresses': {}}, 'annotations': {}}}, 'build_commands': [], 'base_spec': {'apiVersion': 'nuclio.io/v1', 'kind': 'Function', 'metadata': {'annotations': {'nuclio.io/generated_by': 'function generated at 23-10-2019 

In [7]:
addr = fn.deploy(project='nuclio-serving')

[nuclio] 2019-10-23 07:46:18,099 (info) Building processor image
[nuclio] 2019-10-23 07:46:26,183 (info) Build complete
[nuclio] 2019-10-23 07:46:32,248 (info) Function deploy complete
[nuclio] 2019-10-23 07:46:32,254 done updating iris-sklearn-advanced, function address: 192.168.206.37:31664


In [8]:
import requests

# Get sample event 
samples = [[6.4, 2.8, 5.6, 2.2], [5.2, 3.4, 1.4, 0.2]]

# Seldon protocol event
event_seldon = {"data": {"ndarray": samples}}
event_general = {"data": samples}
even_tensorflow = {"instances": samples}
event = event_seldon

csel = str(event).replace("\'", "\"")
print(f'Sent event:\n{csel}\n')

resp = requests.put(addr + '/predict/iris_v1', data=csel)
print(f'Prediction (Response):\n{resp.content}')

Sent event:
{"data": {"ndarray": [[6.4, 2.8, 5.6, 2.2], [5.2, 3.4, 1.4, 0.2]]}}

Prediction (Response):
b'{"data": {"ndarray": [2, 0]}}'
